# Generate point-in-time best execution data using dedicated service - C#

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `point-in-time` approach and designed to highlight individual trades within market activity over a look back period. 

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the interval approach, suitable to analyze market activity over a look back period. Please refer to <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *point-in-time* results.

### Inputs/outputs
Best execution PIT sample requires instrument's identifier, **single trade** and time window as per inputs. Results are as follows:
* a snapshot of occured trades around the given trade based on the input time window

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***


# Run BestEx Point-in-time sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Instrument selection

In [ ]:
// Set the instrument  identifier: a ticker and an exchange code
var ticker = "AAPL";
var exchange = "BATS";

// Set the time window to delimit the input trade
var window = TimeSpan.FromMinutes(15);

#### 3.2 Trade parameters

We assume our trade is the following:

In [ ]:
var timestamp = Convert.ToDateTime("2021-03-05T12:13:21.877300Z");
var price = 119.81;
var size = 100;

#### 3.3 Time period delimitation

In [ ]:
// Build the tick trades request date interval (we are using Google date time format)
var dateIntervals = new DateInterval(){ 
    StartDate = new Date { Year = timestamp.Year, Month = timestamp.Month, Day = timestamp.Day },
    EndDate = new Date { Year = timestamp.Year, Month = timestamp.Month, Day = timestamp.Day } 
};

// Get the first and the last times from the input trade
var firstTime = timestamp.TimeOfDay.Add(-window);
var lastTime = timestamp.TimeOfDay.Add(window);

// Build the tick trades request time interval (we are using Google date time format)
// UTC time zone
var timeInterval = new TimeInterval() { 
    StartTime = new TimeOfDay { Hours = firstTime.Hours, Minutes = firstTime.Minutes, Seconds = firstTime.Seconds }, 
    EndTime = new TimeOfDay { Hours = lastTime.Hours, Minutes = lastTime.Minutes, Seconds = lastTime.Seconds }
};

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Generate constraints based on the previous time selection
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

In [ ]:
// Generate the tick trades request
var request = new TickTradesRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [ ]:
// Instantiate the tick trades service
var service = new TickTradesService.TickTradesServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrive trades

In [ ]:
// Get the trades 
// store the list of trades
var trades = new List<Tuple<Timestamp, Trade>>();

// Call the service
var call = service.TickTrades(request, headers);

// Process the responses
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    if(current.Mapping != null)
    {
        // First response contains the mapping fields
        // Skip the mapping data
        continue;
    }

    // Get the time stamp for the current trade
    var ts = current.Data.TimeStamp;

    // Trade
    if (current.Data.Trade != null) 
    {      
        trades.Add(new Tuple<Timestamp, Trade>(ts, current.Data.Trade));
    }
}

The following code snippet displays request results:

In [ ]:
// Display the trades
//--> Note : the time stamp is displayed by default (using the unix format)
//--> We can use other displayers for different output formats 
//--> For more details about the time stamp: https://developers.google.com/protocol-buffers/docs/reference/google.protobuf#google.protobuf.Timestamp 
//display(trades);

#### 4.2 Plot individual trade within market snapshot
Display trade prices and highlight the individual trade over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graph with the trade prices and mark out the individual trade as follows: 

In [ ]:
// Create Trade class to handle the input trades
public class Input
{
    public DateTime Timestamp { get; set; }
    public double Price { get; set; }
    public long Size { get; set; }
}

// Input trade : trade to validate
var input = new Input() { Timestamp = timestamp, Price = price, Size = size };
var inputs = new List<Input>(){input};

In [ ]:
var bestexPit = new List<Scattergl>
{
    new Scattergl
    { name = "Price", x = trades.Select(elt => elt.Item1.ToDateTime()), y = trades.Select(elt => elt.Item2.Price), mode = "lines" },
    new Scattergl
    { name = "Input trade price", x = inputs.Select(t => t.Timestamp), y = inputs.Select(t => t.Price), mode = "markers" },
    new Scattergl
    { name = "Input trade size", x = inputs.Select(t => t.Timestamp), y = inputs.Select(t => t.Size), mode = "markers", yaxis = "y2" },
    new Scattergl
    { name = "Size", mode = "lines", x = trades.Select(elt => elt.Item1.ToDateTime()), y = trades.Select(elt => elt.Item2.Size), yaxis = "y2"}
};

var layout = new Layout.Layout
{ 
    title = $"Input trade within market snapshot for {ticker}-{exchange}",
    yaxis = new Yaxis { title = "Price", autorange = true, side = "left"},
    yaxis2 = new Yaxis { title = "Size", autorange = true, side = "right"}
};

var chart = Chart.Plot(bestexPit, layout);
chart.WithXTitle("Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);